Notwendige Pakete

In [21]:
### Ziel: Installieren der notwendigen Pakete und Einrichtung der Umgebung ###

#Empfehlung: Neue Conda-Umgebung einrichten
#!conda create --name beng python=3.12.0
#!conda activate beng

#%pip install requests
#%pip install xml.etree.ElementTree
#%pip install --upgrade --quiet langchain langchain-openai
#%pip install python-dotenv
#%pip install translate
#%pip install reportlab
#%pip install pdfkit
#%pip install lxml



Zugriff DEXPI-R&I 

In [22]:
### Ziel: Zugriff auf das DEXPI-R&I im .xml-Format herstellen ###
# Quelle GitHub Repositry der Bachelorarbeit: https://github.com/FloT10/MillerFlorinBA

# Importieren des 'requests'-Moduls, um Daten von einer URL herunterzuladen, und das 'xml.etree.ElementTree'-Modul, um XML-Daten zu verarbeiten.
import requests
import xml.etree.ElementTree as ET

# GitHub Raw-URLs für die DEXPI-Datei.
github_url_dexpi = 'https://raw.githubusercontent.com/FloT10/MillerFlorinBA/main/C01V01-HEX.EX03.xml'
# Im aktuellen Entwurf nicht nötig.
# github_url_formblatt = 'https://github.com/FloT10/MillerFlorinBA/blob/546c3688eccc25c23648de29965ede3a476fcb0b/formblatt_6_2.xml'

# Herunterladen der DEXPI-Datei von der GitHub-URL.
response = requests.get(github_url_dexpi)

# Speichern der heruntergeladenen DEXPI-Datei lokal im Textmodus ('w').
with open('/tmp/C01V01-HEX.EX03.xml', 'w') as file:
    file.write(response.text)

# Definieren eines Dateipfades zur lokal gespeicherten DEXPI-Datei zur weiteren Verwendung.
dexpi_xml = "/tmp/C01V01-HEX.EX03.xml"

with open('/tmp/C01V01-HEX.EX03.xml', 'r') as file:
    dexpi_xml_test = file.read(1000)
    print(dexpi_xml_test)

<?xml version="1.0" encoding="utf-8"?>
<PlantModel xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="../ProteusPIDSchema_4.0.1.xsd">
  <!--Created with INGR ISO15926 PostProc V0.0.7.1 and ConfigurationFileVersion 1.1 by PPM-VM-2014-R1\Intergraph on PPM-VM-2014-R1 at 4/12/2018 5:14:33 PM-->
  <PlantInformation SchemaVersion="4.0.1" OriginatingSystem="SPPID" Date="2018-04-12" Time="15:13:38.0000000+02:00" Is3D="no" Units="Metre" Discipline="PID">
    <UnitsOfMeasure Distance="Metre" />
  </PlantInformation>
  <PlantStructureItem ID="PBS945900F29F424D32A292F07BB90792E2" ComponentClass="PlantSectionIso10209-2012" ComponentName="Github" ComponentClassURI="http://sandbox.dexpi.org/rdl/PlantSectionIso10209-2012">
    <GenericAttributes Number="2" Set="DexpiAttributes">
      <GenericAttribute Name="PlantSectionIdentificationCodeAssignmentClass" Value="Github" Format="string" AttributeURI="http://sandbox.dexpi.org/rdl/PlantSectionIdentificationCodeAssignment


Angaben zur Anlage

In [23]:
### Ziel: Extrahieren der Anlagenbezeichnung ###

# Funktion zum Laden und Extrahieren von Daten aus der DEXPI-Datei.
def extrahiere_anlagenbezeichnung(dexpi_xml):
    global anlagenbezeichnung
    try:
        # Laden des XML-Dokuments.
        tree = ET.parse(dexpi_xml)
        root = tree.getroot()

        # Extrahiere die Anlagenbezeichnung aus dem XML-Baum.
        drawing_element = root.find('.//Drawing')
        if drawing_element is not None:
            anlagenbezeichnung = drawing_element.get('Name')

        else:
            print("Die Anlagenbezeichnung wurde nicht gefunden.")

    except Exception as e:
        # Falls ein Fehler auftritt, gib eine Fehlermeldung aus.
        print(f"Fehler beim Laden und Extrahieren der Daten: {e}")

# Beispielaufruf der Funktion
extrahiere_anlagenbezeichnung(dexpi_xml)

# Ausgabe der Variable.
print(anlagenbezeichnung)


C01V01-HEX.EX03


In [24]:
### Ziel: Extrahieren des Anlagenumfangs ###

# Funktion zum Laden und Extrahieren von Daten aus der DEXPI-Datei.
def extrahiere_anlagenumfang(dexpi_xml):
    try:
        # Lade das XML-Dokument und erhalte die Wurzel des Baums.
        tree = ET.parse(dexpi_xml)
        root = tree.getroot()

        # Extrahiere ComponentClass aus allen Equipment- und PipingNetworkSystem-Elementen.
        equipment_elements = root.findall('.//Equipment')
        piping_network_system_elements = root.findall('.//PipingNetworkSystem')

        component_classes = set()

        for equipment_element in equipment_elements:
            component_class = equipment_element.get('ComponentClass')

            # Überprüfe, ob die ComponentClass den erlaubten Werten entspricht.
            # Ausgerichtet am Beispiel-RI. Andere Anlagenbestandteile könnten ergänzt werden.
            erlaubte_component_classes = ['Tank', 'ReciprocatingPump', 'CentrifugalPump', 'PlateAndShellHeatExchanger', 'ShellAndTubeHeatExchanger']
            if component_class in erlaubte_component_classes:
                component_classes.add(component_class)

        for piping_network_system_element in piping_network_system_elements:
            component_class = piping_network_system_element.get('ComponentClass')

            # Überprüfe, ob die ComponentClass den erlaubten Werten entspricht.
            erlaubte_component_classes = ['PipingNetworkSystem']
            if component_class in erlaubte_component_classes:
                component_classes.add(component_class)

        # Überprüfe, ob mindestens eine gültige ComponentClass gefunden wurde.
        if not component_classes:
            raise ValueError("Keine gültigen Anlagenbestandteile gefunden.")

        # Weitere Werte können entsprechend extrahiert werden.

        # Gib ein Dictionary mit den extrahierten Werten zurück.
        return {
            'Anlagenumfang (englisch)': list(component_classes),
            # Weitere extrahierte Werte könnten hier hinzugefügt werden.
        }

    except Exception as e:
        # Falls ein Fehler auftritt, gib eine Fehlermeldung aus und gibt None zurück.
        print(f"Fehler beim Laden und Extrahieren der Daten: {e}")
        return None

# Beispielaufruf der Funktion und Ausgabe der extrahierten Daten.
anlagenumfang_englisch = extrahiere_anlagenumfang(dexpi_xml)
# print(anlagenumfang_englisch)

# Übersetzung des Amlagenumfangs von english zu deutsch
    # Ggf. muss Tabelle erweitert werden
def uebersetze_begriffe(begriffe, zielsprache='de'):
    manuelle_uebersetzungen = {
        'ReciprocatingPump': 'Kolbenpumpe',
        'PipingNetworkSystem': 'Rohrleitungssystem',
        'CentrifugalPump': 'Zentrifugalpumpe',
        'PlateAndShellHeatExchanger': 'Plattenwärmetauscher',
        'Tank': 'Behälter',
        'ShellAndTubeHeatExchanger': 'Rohrbündelwärmetauscher'
    }

    uebersetzte_begriffe = [manuelle_uebersetzungen.get(begriff, begriff) for begriff in begriffe]
    return uebersetzte_begriffe

# Beispielaufruf
anlagenumfang_liste = uebersetze_begriffe(anlagenumfang_englisch['Anlagenumfang (englisch)'])

# Verbinden der Elemente der Liste zu einem String mit Kommas
anlagenumfang = ", ".join(anlagenumfang_liste)


# Ausgabe der übersetzten Begriffe
print(anlagenumfang)



Rohrleitungssystem, Plattenwärmetauscher, Rohrbündelwärmetauscher, Kolbenpumpe, Zentrifugalpumpe, Behälter


In [25]:
### Ziel: Extrahieren der Anlagenart ###

def finde_groessten_tank(dexpi_xml):
    try:
        # Parsen der DEXPI-Datei
        tree = ET.parse(dexpi_xml)
        root = tree.getroot()
    except ET.ParseError as e:
        # Fehlerausgabe, wenn das Parsen fehlschlägt
        print("Fehler beim Parsen des XML-Dokuments:", e)
        return None, None
    
    # Initialisierung von Variablen für den größten Tank und dessen Kapazität
    groesster_tank = None
    max_capacity = 0
    
    # Iterieren über alle 'Equipment'-Elemente im XML-Dokument
    for tank in root.iter('Equipment'):
        if tank.attrib.get('ComponentClass') == 'Tank':
            # Suche nach dem Attribut 'NominalCapacity(Volume)' im aktuellen Tank
            capacity_attr = tank.find(".//GenericAttribute[@Name='NominalCapacity(Volume)']")
            
            # Überprüfen, ob das Attribut gefunden wurde und einen 'Value'-Schlüssel hat
            if capacity_attr is not None and 'Value' in capacity_attr.attrib:
                # Extrahieren der Kapazität und vergleichen mit der bisherigen maximalen Kapazität
                capacity = float(capacity_attr.attrib['Value'])
                if capacity > max_capacity:
                    max_capacity = capacity
                    groesster_tank = tank
    
    # Wenn ein größerer Tank gefunden wurde
    if groesster_tank is not None:
        # Extrahieren der Tag-Bezeichnung des größten Tanks
        tank_tag = groesster_tank.attrib['TagName']
        
        # Extrahieren der Beschreibungen der Kammern im größten Tank
        tank_beschreibungen = [chamber.find(".//GenericAttribute[@Name='ChamberFunctionAssignmentClass']").attrib.get('Value', '') for chamber in groesster_tank.iter('Equipment') if chamber.attrib.get('ComponentClass') == 'Chamber']
        
        return groesster_tank, tank_tag, tank_beschreibungen
    else:
        # None, wenn kein größerer Tank gefunden wurde
        return None, None

# Aufruf mit XML-Dokument
groesster_tank, tank_tag, tank_beschreibungen = finde_groessten_tank(dexpi_xml)

# Ausgabe der Ergebnisse
#print("Größter Tank:", tank_tag)
#print("Beschreibungen der Kammern:", tank_beschreibungen)

def setze_anlagenart(tank_beschreibungen):
    # Überprüfen, ob 'Processing Chamber' in den Beschreibungen vorkommt
    if "Processing" in tank_beschreibungen:
        return "HBV-Anlage"
    # Überprüfen, ob 'Storage Chamber' in den Beschreibungen vorkommt
    elif "Storing" in tank_beschreibungen:
        return "Tanklager"
    else:
        # Fehlermeldung, wenn keine oder beide Beschreibungen vorhanden sind
        print("Fehler: Ungültige Anlagenkonfiguration. Anlagenart manuell eintragen.")
        return None

# Beispielaufruf
anlagenart = setze_anlagenart(tank_beschreibungen)

# Ausgabe der Ergebnisse
print(anlagenart)



HBV-Anlage


Angaben zu den wassergefährdenden Stoffen in der Anlage

In [26]:
### Ziel: Extrahieren der Stoffe ###

#Alle Pumpen und Behälter selektieren mit allen Werten und dann Auswahl welche gewertet werden sollen

In [40]:
### Zwischenschritt zum Testen ###
### Ziel: Verbindung zwischen Pumpe und vorgeschalteter Rohrleitung herstellen um das geführte Medium zu ermitteln###
    # test_stutzen_id = ID des Eingangsstutzens der Pumpe
    # rls_id = ID des Rohrleitungssegments vor der Pumpe  

def find_piping_network_segment(dexpi_xml, test_stutzen_id):
    # Parsen der DEXPI-Datei
    tree = ET.parse(dexpi_xml)
    root = tree.getroot()

    # Iteriere über alle PipingNetworkSegment-Elemente
    for piping_network_segment in root.iter('PipingNetworkSegment'):
        # Iteriere über alle Connection-Elemente unter dem aktuellen PipingNetworkSegment
        for connection in piping_network_segment.iter('Connection'):
            to_id = connection.get("ToID")
            if to_id == test_stutzen_id:
                # Die ID wurde gefunden, nun die übergeordnete Piping-Network-Segment-ID abrufen
                piping_network_segment_id = piping_network_segment.get("ID")
                return piping_network_segment_id

    # Wenn die ID nicht gefunden wurde, gib eine entsprechende Meldung aus
    print(f"Kein Piping-Network-Segment mit der ToID {test_stutzen_id} gefunden.")
    return None

# Beispielaufruf
test_stutzen_id = "SP5DE5B0281BC74D0AB470383D03E679CC"
rls_id = find_piping_network_segment(dexpi_xml, test_stutzen_id)
if rls_id:
    print(f"Die Piping-Network-Segment-ID für die Eingangsstutzen-Pumpe-ID {test_stutzen_id} ist: {rls_id}")




Die Piping-Network-Segment-ID für die Eingangsstutzen-Pumpe-ID SP5DE5B0281BC74D0AB470383D03E679CC ist: SGF20C52DAB81A42D19331F749B484A9D9


In [54]:
### Ziel: Verbindung zwischen Pumpe und vorgeschalteter Rohrleitung herstellen um das geführte Medium zu ermitteln###
    # nozzle_id = ID des Eingangsstutzens der Pumpe
    # Ausgegebende IN 

def extrahiere_medium_pumpe(dexpi_xml, nozzle_id):
    try:
        tree = ET.parse(dexpi_xml)
        root = tree.getroot()
    except ET.ParseError as e:
        print(f"Fehler beim Parsen des XML: {e}")
        return None

    # Iteriere über alle PipingNetworkSegment-Elemente
    for piping_network_segment in root.iter('PipingNetworkSegment'):
        # Iteriere über alle Connection-Elemente unter dem aktuellen PipingNetworkSegment
        for connection in piping_network_segment.iter('Connection'):
            to_id = connection.get("ToID")
            if to_id == nozzle_id:
                # Die ToID wurde gefunden, nun die Fluid Code Assignment Class abrufen
                for generic_attributes in piping_network_segment.iter('GenericAttributes'):
                    if generic_attributes.get("Set") == "DexpiAttributes":
                        for generic_attribute in generic_attributes.iter('GenericAttribute'):
                            if generic_attribute.get("Name") == "FluidCodeAssignmentClass":
                                fluid_code_assignment_class = generic_attribute.get("Value")
                                return fluid_code_assignment_class

    print(f"Keine Fluid Code Assignment Class für die ToID {nozzle_id} gefunden.")
    return None

# Beispielaufruf
medium_pumpe = extrahiere_medium_pumpe(dexpi_xml, nozzle_id)

def find_pumps_and_nozzle_ids(dexpi_xml):
    try:
        # Parsen der DEXPI-Datei
        tree = ET.parse(dexpi_xml)
        root = tree.getroot()
    except ET.ParseError as e:
        # Fehlerausgabe, wenn das Parsen fehlschlägt
        print("Fehler beim Parsen des XML-Dokuments:", e)
        return

    # Iterieren über alle 'Equipment'-Elemente im XML-Dokument
    for pump in root.iter('Equipment'):
        if 'Pump' in pump.attrib.get('ComponentClass', ''):
            # Suche nach dem Attribut 'DesignVolumeFlowRate' im aktuellen Pumpen-Equipment
            volume_flow_rate_attr = pump.find(".//GenericAttribute[@Name='DesignVolumeFlowRate']")
            
            # Überprüfen, ob das Attribut gefunden wurde und einen 'Value'-Schlüssel hat
            if volume_flow_rate_attr is not None and 'Value' in volume_flow_rate_attr.attrib:
                # Extrahieren der Designvolumenstromrate
                design_volume_flow_rate = float(volume_flow_rate_attr.attrib['Value'])
                pump_id = pump.attrib['ID']

                # Ausgabe für die Pumpe
                print(f"Pumpe mit ID: {pump_id}")
                print(f"Volumenstrom: {design_volume_flow_rate} m³/h")

                # Suche nach der ID des Nozzles 'N1' für jede Pumpe
                nozzle_id = pump.find(".//Nozzle[@TagName='N1']")
                if nozzle_id is not None:
                    nozzle_id = nozzle_id.attrib.get('ID', '')
                    print(f"Stutzen ID: {nozzle_id}")

                    # Jetzt rufen wir die Funktion auf, um den Fluid Code zu finden
                    fluid_code = extrahiere_medium_pumpe(dexpi_xml, nozzle_id)
                    if fluid_code:
                        print(f"Fluid Code: {fluid_code}")
                    else:
                        print("Kein Fluid Code gefunden.")

                # Leerzeile zwischen den Pumpe-Informationen
                print()

# Beispielaufruf
find_pumps_and_nozzle_ids(dexpi_xml)



Pumpe mit ID: SP570CBE2AF8344DD29AF899EE341A7808
Volumenstrom: 420.0 m³/h
Stutzen ID: SP5DE5B0281BC74D0AB470383D03E679CC
Fluid Code: MNc

Pumpe mit ID: SPBDD98F7C78B149F7BDA05772123D0388
Volumenstrom: 200.0 m³/h
Stutzen ID: SP39A8AAF30B144BEE8F4C8428E42D2871
Fluid Code: MNb



Angaben zur Gefährdungsstufe

In [ ]:
### Ziel: Ermittlung der Gefährdungsstufe ###



Angaben zu Behältern

In [ ]:
### Ziel: Extrahieren der Tank-Daten ###
# Im DEXPI-R&I gibt es nur einen Tank. Bei mehreren Tanks sind entsprechende Anpassungen an mehreren Stellen im Code zu prüfen.
# Im DEXPI-R&I fehlen einige Informationen um die entsprechenden Angaben sauber aus dem Fließbild zu extrahieren. Es wurden Informationen in der dexpi_xml ergänzt.

def extrahiere_tanks(dexpi_xml):
    
    # Parsen der dexpi_xml und Erstellen des Wurzelelements
    root = ET.parse(dexpi_xml)
    tanks = []

    # Durchlaufen aller Equipment-Elemente mit ComponentClass='Tank'
    for equipment in root.findall(".//Equipment[@ComponentClass='Tank']"):
        # Extrahieren des Tag-Namens des Tanks
        tag_name = equipment.get('TagName', '')

        # Aussortieren von Tanks ohne Tag-Namen, die mit 'T' beginnen
        if not tag_name.startswith('T'):
            continue

        # Extrahieren des NominalCapacity-Elements
        # Sowohl DEXPI-R&I als auch das Formblatt sind auf m3 ausgelegt. An dieser Stelle könnte zusätzlich der Umgang mit unterschiedlichen Einheiten vorgesehen werden. 
        nominal_capacity_element = equipment.find(".//GenericAttribute[@Name='NominalCapacity(Volume)']")
        
        # Überprüfen, ob NominalCapacity-Element vorhanden ist
        if nominal_capacity_element is not None:
            # Extrahieren des Wertes von NominalCapacity
            nominal_capacity_str = nominal_capacity_element.get('Value', '')

            # Überprüfen, ob NominalCapacity eine Zahl ist
            try:
                nominal_capacity = float(nominal_capacity_str)
            except ValueError:
                # Fehlermeldung ausgeben und zum nächsten Tank gehen
                print(f"Fehler: Nennvolumen für Tank mit Tanknummer '{tag_name}' ist keine Zahl.")
                continue
        else:
            # Fehlermeldung ausgeben und zum nächsten Tank gehen
            print(f"Fehler: Kein Nennvolumen für Tank mit Tanknummer '{tag_name}' vorhanden.")
            continue

        # Extrahieren des FluidCodeAssignmentClass-Elements
        fluid_code_element = equipment.find(".//GenericAttribute[@Name='FluidCodeAssignmentClass']")
        
        # Überprüfen, ob FluidCodeAssignmentClass-Element vorhanden ist
        if fluid_code_element is not None:
            fluid_name = fluid_code_element.get('Value', '')
        else:
            # Fehlermeldung ausgeben und zum nächsten Tank gehen
            print(f"Fehler: Kein FluidCodeAssignmentClass für Tank mit Tanknummer '{tag_name}' vorhanden.")
            continue

        # Überprüfen, ob es Kammern gibt
        chambers = equipment.findall(".//Equipment[@ComponentClass='Chamber']")
        
        # Bestimmen der Tank-Wandigkeit
            # Als Standard ist einwandig gesetzt. Sollte es am Tank einen Überwachungsraum zur Leckagedetection geben, wäre der Tank doppelwandig (§2(17) AwSV)
        # Bestimmen des Tank-Materials
            # Im DEXPI-R&I hat der Tank selbst kein Material, aber seine Kammern. In der Praxis sind unterschiedliche Materialien an einem Tank durchaus möglich. Hier wird die "Hauptkammer" analysiert.


        chamber_function = "einwandig"
        chamber_materials = set()

        for chamber in chambers:
            chamber_function_value = chamber.find(".//GenericAttribute[@Name='ChamberFunctionAssignmentClass']").get('Value', '')
            
            if chamber_function_value in ['Processing', 'Storing']:
                # Extrahieren des Materials der Kammer
                chamber_material_element = chamber.find(".//GenericAttribute[@Name='MaterialOfConstructionCodeAssignmentClass']")
                if chamber_material_element is not None:
                    chamber_material = chamber_material_element.get('Value', '')
                    chamber_materials.add(chamber_material)
                else:
                    # Fehlermeldung ausgeben und zum nächsten Tank gehen
                    print(f"Fehler: Kein MaterialOfConstructionCodeAssignmentClass für Kammer im Tank mit Tanknummer '{tag_name}' vorhanden.")
                    continue

                # Falls es eine Kammer mit "leakage detection" gibt, ist der Tank doppelwandig
                # Leakage detection wäre als ClassOfActivity definiert: 
                if chamber_function_value == "leakage detection":
                    chamber_function = "doppelwandig"

        # Meldung, wenn keine oder mehrere solcher Kammern gefunden wurden
        if len(chamber_materials) == 0:
            print(f"Warnung: Keine Kammern mit den Funktionen 'Processing' oder 'Storage' für Tank mit Tanknummer '{tag_name}' gefunden.")
        elif len(chamber_materials) > 1:
            print(f"Warnung: Mehrere Kammern mit den Funktionen 'Processing' oder 'Storage' für Tank mit Tanknummer '{tag_name}' gefunden. Es wird nur das Material der ersten Kammer verwendet.")

        # Extrahieren des Engineering Standard
        engineering_standard_element = equipment.find(".//GenericAttribute[@Name='EngineeringStandardClass']")
        engineering_standard = engineering_standard_element.get('Value') if engineering_standard_element is not None else None

        # Zusammenstellen der Tankdaten und Hinzufügen zur Liste
        tank_data = {
            'tank_nummer': tag_name,
            'tank_volumen': nominal_capacity,
            'tank_stoff': fluid_name,
            'tank_wandig': chamber_function,
            'tank_material': chamber_materials.pop() if chamber_materials else None,
            'tank_zulassung': engineering_standard,
        }
        tanks.append(tank_data)

    # Rückgabe der Liste von Tanks
    return tanks

# Aufrufen der Funktion mit der DEXPI-XML-Datei und Ausgabe der Ergebnisse
tanks = extrahiere_tanks(dexpi_xml)

for tank in tanks:
    print(f'{tank["tank_nummer"]}, {tank["tank_stoff"]}, {tank["tank_wandig"]}, {tank["tank_volumen"]}, {tank["tank_material"]}, , {tank["tank_zulassung"]}')


T4750, MNb, einwandig, 22.0, 1.4306, , AD2000


KI-Anwendung Material
Quelle: https://python.langchain.com/docs/use_cases/chatbots/quickstart

In [ ]:
# Importieren der benötigten Module
# import os

# Extrahieren des OpenAI-API-Schlüssels aus den Umgebungsvariablen oder direkt eintragen zwischen ""
# openai_api_key = os.getenv("OPENAI_API_KEY", "")

# Jetzt ist der OpenAI-API-Schlüssel verfügbar und kann im Code verwendet werden, der print unten nur zum Test
# print(f"OpenAI API Key: {openai_api_key}")

# Importieren der benötigten Module
# from langchain_openai import ChatOpenAI

# Initialisieren des ChatOpenAI-Objekts
# Mit der temperature kann man "herum experimentieren"
# chat = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=1, openai_api_key=openai_api_key)


In [ ]:
# Definition der Werkstoffbezeichnung, hier kein Automatismus
# Es können unterschiedliche Werkstoffe getestet werden
#werkstoff_bezeichnung = "1.4571"

# Verwendung der Variable in der Chat-Anfrage
#from langchain_core.messages import HumanMessage

#chat.invoke(
#    [
#        HumanMessage(
#            content=f'Setze die Sprache auf Deutsch. Du bist ein Assistent. Du hast eine Aufgabe. Ich gebe dir einen Werkstoff. Deine Aufgabe besteht darin, zuzuordnen, ob es sich beim gegebenen Werkstoff um ein Kunststoff, ein Metall oder einen anderes Material handelt. Versuche den gegebenen Werkstoff in eine der drei Kategorien: "Metall", "Kunststoff", "anderes Material" einzustufen. Antworte nur: "Metall", "Kunststoff", "anderes Material".  Der gesuchte Werkstoff ist: {werkstoff_bezeichnung}. Falls es sich dabei gar nicht um einen Werkstoff im engeren Sinn handelt sage "Error"'
#        )
#    ]
#)


Angaben zu den Rohrleitungen

In [ ]:
### Ziel: Beurteilung der relevanten Rohrleitungen ###
    # Eine PipingNetworkSystem soll als eine Rohrleitung gewertet werden
    # Im DEXPI-Viewer wurden 11 Rohrleitungen identifiziert: https://dexpi-viewer.model-broker.com/ 
    # Durch zählen auf PDF R&I werden auch 11 Rohrleitungen identifiziert

# Parsen der XML-Daten
tree = ET.parse(dexpi_xml)
root = tree.getroot()

# Zählen der Piping Network Systems
piping_network_system_count_all = len(root.findall(".//PipingNetworkSystem"))

# Ausgabe der Gesamtanzahl an Piping Network Systems aus der Quelldatei
print(f"Gesamtanzahl Piping Network Systems: {piping_network_system_count_all}")

# Iterieren über alle Piping Network Systems und Extrahieren der IDs und TagNames
for piping_system in root.iter('PipingNetworkSystem'):
    # Extrahieren des TagNames
    piping_system_tagname = piping_system.attrib.get('TagName', '')
    
    # Prüfen, ob der TagName aus genau 5 Ziffern besteht
    if piping_system_tagname.isdigit() and len(piping_system_tagname) == 5:
        # Extrahieren der ID
        piping_system_id = piping_system.attrib.get('ID', '')
    
        # Ausgabe der ID und des TagNames
        print(f"Piping Network System ID: {piping_system_id}, TagName: {piping_system_tagname}")
    else:
        # Ausgabe für ungültigen Tag
        print(f"Piping Network System ID: {piping_system.attrib.get('ID', '')}, Kein gültiger Tag")



Gesamtanzahl Piping Network Systems: 14
Piping Network System ID: SY5D814AA7FC0A47F5921830DCF5A4A8EB, TagName: 47131
Piping Network System ID: SYF0E638DCE25148A0ADE8BE802F8AFAE3, TagName: 47130
Piping Network System ID: SY11933F04339546E2BB3FAE0C30B7D6AA, TagName: 47122
Piping Network System ID: SY043C45C1F8ED42C092CDCC5BF3456CB4, TagName: 47121
Piping Network System ID: SY708551978F43471FA98F8244FC0A913F, TagName: 47140
Piping Network System ID: SYAEBFFC9AB2C94F5993EA8C4D89EC1125, TagName: 47126
Piping Network System ID: SY1F37391ED2314597A4A128606FDD1B01, TagName: 47125
Piping Network System ID: SY8B12D6816E9F432695222500F08A648F, TagName: 47124
Piping Network System ID: SY81C7023B2F9846A4B5AB695A2DDB2140, Kein gültiger Tag
Piping Network System ID: SYE2E35A8ADC204BBEA377E7EBFF661EB6, Kein gültiger Tag
Piping Network System ID: SYAC2C1AB5FF9F4AE59D88F023714689A1, Kein gültiger Tag
Piping Network System ID: SY1D329D8EFB14468CB0F4807D9312F472, TagName: 47123
Piping Network System ID: S

In [ ]:
### Ziel: Kategorisierung und Aufsummierung der relevanten Rohrleitungen ###
### Alternativ: Bearbeitung XML und andere Klassen bestimmen. Z.B.: SUCTION PIPING SYSTEM (http://data.posccaesar.org/rdl/RDS426402071)

# Parsen der XML-Daten
tree = ET.parse(dexpi_xml)
root = tree.getroot()

# Piping Network Systems, die ausgeschlossen werden sollen, da Werte nicht sinnvoll erscheinen (kein gültiger Tag usw.)
# Manueller Eingriff aufgrund mangelnder Datenqualität in Quelldatei
excluded_system_ids = ['SYAC2C1AB5FF9F4AE59D88F023714689A1', 'SYE2E35A8ADC204BBEA377E7EBFF661EB6', 'SY81C7023B2F9846A4B5AB695A2DDB2140']

# Zählen der verbleibenden  Piping Network Systems (mit Ausschluss)
piping_network_systems = [piping_system for piping_system in root.iter('PipingNetworkSystem') if piping_system.attrib.get('ID') not in excluded_system_ids]
piping_network_system_count = len(piping_network_systems)

# Initialisierung von Zählern für die manuell zugewiesenen Kategorien
einwandig_standard_count = 0
einwandig_saugleitung_count = 0
einwandig_schutzrohr_count = 0
doppelwandig_count = 0

# Initialisierung von Listen für die Tags pro Kategorie
einwandig_standard_tags = []
einwandig_saugleitung_tags = []
einwandig_schutzrohr_tags = []
doppelwandig_tags = []

def kategorisiere_rohrleitungen(piping_system):
    jacketed_pipe_specialization = piping_system.find(".//GenericAttribute[@Name='JacketedPipeSpecialization']")
    
    if jacketed_pipe_specialization is not None:
        specialization_value = jacketed_pipe_specialization.attrib.get('Value', '')
        
        if specialization_value.lower() == 'unjacketedpipe':
            category_input = input(f"Bitte Kategorie für Rohrleitung {piping_system.attrib.get('TagName')} (Enter für Einwandig Standard, 'saug' für Einwandig Saugleitung, 'schutz für Einwandig Schutzrohr): ").strip().lower()

            if category_input == '':
                return 'Einwandig Standard'
            elif category_input == 'saug':
                return 'Einwandig Saugleitung'
            elif category_input == 'schutz':
                return 'Einwandig Schutzrohr'
            else:
                print(f"Ungültige Eingabe: {category_input}. Einwandig Standard wird ausgewählt.")
                return 'Einwandig Standard'
        elif specialization_value.lower() == 'jacketedpipe':
            return 'Doppelwandig'
        else:
            return "Ungültige Piping Network System-Kategorie gefunden."
    else:
        return "Kein unjacketed Piping Network System gefunden."

# Verwendung der Funktion in Ihrer Schleife
for piping_system in piping_network_systems:
    result = kategorisiere_rohrleitungen(piping_system)
    
    # Zählvariablen und Listen aktualisieren
    if result == 'Einwandig Standard':
        einwandig_standard_count += 1
        einwandig_standard_tags.append(piping_system.attrib.get('TagName'))
    elif result == 'Einwandig Saugleitung':
        einwandig_saugleitung_count += 1
        einwandig_saugleitung_tags.append(piping_system.attrib.get('TagName'))
    elif result == 'Einwandig Schutzrohr':
        einwandig_schutzrohr_count += 1
        einwandig_schutzrohr_tags.append(piping_system.attrib.get('TagName'))
    elif result == 'Doppelwandig':
        doppelwandig_count += 1
        doppelwandig_tags.append(piping_system.attrib.get('TagName'))
    
    # Ergebnis ausgeben
    print(f"Kategorie für Rohrleitung {piping_system.attrib.get('TagName')}: {result}")

# Ausgabe der Ergebnisse
print("\n# Ausgabe der Anzahl der Rohrleitungen")
print(f"Anzahl Gesamt: {piping_network_system_count}")
print(f"Anzahl Doppelwandig: {doppelwandig_count}")
print(f"Anzahl Einwandig Standard: {einwandig_standard_count}")
print(f"Anzahl Einwandig Saugleitung: {einwandig_saugleitung_count}")
print(f"Anzahl Einwandig Schutzrohr: {einwandig_schutzrohr_count}")

# Ausgabe der Tags pro Kategorie
print("\n# Ausgabe der Tag-Zuordnung")
print(f"Tags für Doppelwandig: {doppelwandig_tags}")
print(f"Tags für Einwandig Standard: {einwandig_standard_tags}")
print(f"Tags für Einwandig Saugleitung: {einwandig_saugleitung_tags}")
print(f"Tags für Einwandig Schutzrohr: {einwandig_schutzrohr_tags}")



Kategorie für Rohrleitung 47131: Einwandig Standard
Kategorie für Rohrleitung 47130: Einwandig Standard
Kategorie für Rohrleitung 47122: Einwandig Standard
Kategorie für Rohrleitung 47121: Einwandig Standard
Kategorie für Rohrleitung 47140: Einwandig Standard
Kategorie für Rohrleitung 47126: Einwandig Standard
Kategorie für Rohrleitung 47125: Einwandig Saugleitung
Kategorie für Rohrleitung 47124: Einwandig Schutzrohr
Ungültige Eingabe: ölkj. Einwandig Standard wird ausgewählt.
Kategorie für Rohrleitung 47123: Einwandig Standard
Kategorie für Rohrleitung 47127: Einwandig Saugleitung
Kategorie für Rohrleitung 47141: Einwandig Standard

# Ausgabe der Anzahl der Rohrleitungen
Anzahl Gesamt: 11
Anzahl Doppelwandig: 0
Anzahl Einwandig Standard: 8
Anzahl Einwandig Saugleitung: 2
Anzahl Einwandig Schutzrohr: 1

# Ausgabe der Tag-Zuordnung
Tags für Doppelwandig: []
Tags für Einwandig Standard: ['47131', '47130', '47122', '47121', '47140', '47126', '47123', '47141']
Tags für Einwandig Saugleitun

In [ ]:
# Initialisierung der Listen für die Piping Class Codes
einwandig_standard_classcodes = []
einwandig_saugleitung_classcodes = []
einwandig_schutzrohr_classcodes = []
doppelwandig_classcodes = []

def sammle_rohrklassen(tag_list, piping_network_systems):
    result_list = []

    for piping_system in piping_network_systems:
        tag_name = piping_system.attrib.get('TagName')

        # Nur für Systeme in den ausgewählten Tags
        if tag_name in tag_list:
            # Prüfen der PipingClassCodeAssignmentClass
            class_assignment = piping_system.find(".//GenericAttribute[@Name='PipingClassCodeAssignmentClass']")

            if class_assignment is not None:
                class_code = class_assignment.attrib.get('Value', '').strip()

                # Piping Class Code zur entsprechenden Liste hinzufügen
                result_list.append(class_code)

    return result_list

# Verwendung der Funktion
einwandig_standard_classcodes = sammle_rohrklassen(einwandig_standard_tags, piping_network_systems)
einwandig_saugleitung_classcodes = sammle_rohrklassen(einwandig_saugleitung_tags, piping_network_systems)
einwandig_schutzrohr_classcodes = sammle_rohrklassen(einwandig_schutzrohr_tags, piping_network_systems)
doppelwandig_classcodes = sammle_rohrklassen(doppelwandig_tags, piping_network_systems)

# Ergebnisse ausgeben
print("Rohrklassen Einwandig Standard:", einwandig_standard_classcodes)
print("Rohrklassen Einwandig Saugleitung:", einwandig_saugleitung_classcodes)
print("Rohrklassen Einwandig Schutzrohr:", einwandig_schutzrohr_classcodes)
print("Rohrklassen Doppelwandig:", doppelwandig_classcodes)


Rohrklassen Einwandig Standard: ['75HB13', '75HB13', '75HB13', '75HB13', '75HB13', '75HB13', '75HB13', '75HB13']
Rohrklassen Einwandig Saugleitung: ['75HG12', '75HB13']
Rohrklassen Einwandig Schutzrohr: ['75HB13']
Rohrklassen Doppelwandig: []


In [ ]:
# Initialisierung der Listen für die Piping Class Codes und andere Informationen
einwandig_standard_info = []
einwandig_saugleitung_info = []
einwandig_schutzrohr_info = []
doppelwandig_info = []

def sammle_rohrklassen_und_info(tag_list, piping_network_systems):
    result_list = []

    for piping_system in piping_network_systems:
        tag_name = piping_system.attrib.get('TagName')

        # Nur für Systeme in den ausgewählten Tags
        if tag_name in tag_list:
            # Prüfen der PipingClassCodeAssignmentClass
            class_assignment = piping_system.find(".//GenericAttribute[@Name='PipingClassCodeAssignmentClass']")
            nominal_diameter = piping_system.find(".//GenericAttribute[@Name='NominalDiameterNumericalValueRepresentationAssignmentClass']")

            if class_assignment is not None:
                class_code = class_assignment.attrib.get('Value', '').strip()

                # Piping Class Code und andere Informationen zur entsprechenden Liste hinzufügen
                result_list.append({
                    'Tag': tag_name,
                    'PipingClassCode': class_code,
                    'NominalDiameter': nominal_diameter.attrib.get('Value', '').strip() if nominal_diameter is not None else ''
                })

    return result_list

# Verwendung der Funktion
einwandig_standard_info = sammle_rohrklassen_und_info(einwandig_standard_tags, piping_network_systems)
einwandig_saugleitung_info = sammle_rohrklassen_und_info(einwandig_saugleitung_tags, piping_network_systems)
einwandig_schutzrohr_info = sammle_rohrklassen_und_info(einwandig_schutzrohr_tags, piping_network_systems)
doppelwandig_info = sammle_rohrklassen_und_info(doppelwandig_tags, piping_network_systems)

# Ergebnisse ausgeben
print("Einwandig Standard Informationen:", einwandig_standard_info)
print("Einwandig Saugleitung Informationen:", einwandig_saugleitung_info)
print("Einwandig Schutzrohr Informationen:", einwandig_schutzrohr_info)
print("Doppelwandig Informationen:", doppelwandig_info)


Einwandig Standard Informationen: [{'Tag': '47131', 'PipingClassCode': '75HB13', 'NominalDiameter': '50'}, {'Tag': '47130', 'PipingClassCode': '75HB13', 'NominalDiameter': '50'}, {'Tag': '47122', 'PipingClassCode': '75HB13', 'NominalDiameter': '80'}, {'Tag': '47121', 'PipingClassCode': '75HB13', 'NominalDiameter': '80'}, {'Tag': '47140', 'PipingClassCode': '75HB13', 'NominalDiameter': '50'}, {'Tag': '47126', 'PipingClassCode': '75HB13', 'NominalDiameter': '50'}, {'Tag': '47123', 'PipingClassCode': '75HB13', 'NominalDiameter': '80'}, {'Tag': '47141', 'PipingClassCode': '75HB13', 'NominalDiameter': '50'}]
Einwandig Saugleitung Informationen: [{'Tag': '47125', 'PipingClassCode': '75HG12', 'NominalDiameter': '50'}, {'Tag': '47127', 'PipingClassCode': '75HB13', 'NominalDiameter': '50'}]
Einwandig Schutzrohr Informationen: [{'Tag': '47124', 'PipingClassCode': '75HB13', 'NominalDiameter': '80'}]
Doppelwandig Informationen: []


Rohrklassen definieren 

Logik: Rohrklasse Nenndurchmesser --> Material & Norm 

Formular XML

In [ ]:
### Ziel: Erstellen eines Formblatts 6.2 im .xml-Format ###
# Die anlagenspezifischen Werte welche ins Formblatt eingetragen werden stammen aus den oben ermittelten Variablen.

# XML-Deklaration für die Ausgabe
xml_declaration_str = '<?xml version="1.0" encoding="UTF-8"?>'

# Erstellen des Wurzelelements formblatt_6_2
formblatt_6_2 = ET.Element('formblatt_6_2')
formblatt_6_2.text = xml_declaration_str

def create_table(parent, table_name, heading, column_names, data):
    table = ET.SubElement(parent, f'tabelle_{table_name}')
    table_heading = ET.SubElement(table, 'ueberschrift')
    table_heading.text = heading

    columns = ET.SubElement(table, 'spalten')
    for column_name in column_names:
        column = ET.SubElement(columns, 'spalte')
        column.text = column_name

    table_data = ET.SubElement(table, 'daten')
    data_row = ET.SubElement(table_data, 'zeile')
    for value in data:
        value_element = ET.SubElement(data_row, 'wert')
        value_element.text = value

# Hier können die oben bestimmten Variablen eingesetzt werden 

# Kopftabelle
kopf_data = {
    'Antragsunterlage': 'Für immissionsschutzrechtliches Genehmigungsverfahren',
    'Anlage 1 / Formblatt 6.2': 'Detailangaben / Wassergefährdende Stoffe',
}

# Angaben zur Anlage
angaben_anlage_data = {
    'Anlagenart': anlagenart,
    'Anlagenbezeichnung': anlagenbezeichnung,
    'Anlagenumfang': anlagenumfang
}

# Angaben zu den wassergefährdenden Stoffen in der Anlage
angaben_stoffe_data = {
    'Bezeichnung': 'MNc',
    'Aggregatzustand': 'flüssig',
    'WGK': 'WGK3',
    'Volumen (m3) / Masse (t)': '22'
}

# Angaben zur Gefährdungsstufe
angaben_gefaehrdungsstufe_data = {
    'Maßgebendes Volumen': '22',
    'Maßgebende WGK': 'WGK3',
    'Gefährdungsstufe': 'D'
}

# Angaben zu Behältern
angaben_behaelter_data = {
    'Tanknummer': tank["tank_nummer"],
    'Stoff': tank["tank_stoff"],
    'einwandig / doppelwandig': tank["tank_wandig"],
    'enthaltener Stoff': tank["tank_stoff"],
    'Material': tank["tank_material"],
    'Zulassung': tank["tank_zulassung"]
}

# Angaben zu Rohrleitungen
angaben_rohrleitungen_data = {
    'Bauart': 'Einwandige Rohrleitung',
    'oberirdisch / unterirdisch': 'oberirdisch',
    'Anzahl': '7',
    'Material': 'Metall',
    'Zulassung': 'DIN-Norm'
}


create_table(formblatt_6_2, 'kopf', 'Angaben zum Formblatt', kopf_data.keys(), kopf_data.values())
create_table(formblatt_6_2, 'angaben_anlage', 'Angaben zur Anlage', angaben_anlage_data.keys(), angaben_anlage_data.values())
create_table(formblatt_6_2, 'angaben_stoffe', 'Angaben zu den wassergefährdenden Stoffen', angaben_stoffe_data.keys(), angaben_stoffe_data.values())
create_table(formblatt_6_2, 'angaben_gefäaehrdungsstufe', 'Ermittlung der Gefährdungsstufe der Anlage nach §39 AwSV', angaben_gefaehrdungsstufe_data.keys(), angaben_gefaehrdungsstufe_data.values())
create_table(formblatt_6_2, 'angaben_behaelter', 'Angaben zu Behältern', angaben_behaelter_data.keys(), angaben_behaelter_data.values())
create_table(formblatt_6_2, 'angaben_rohrleitungen', 'Angaben zu Rohrleitungen', angaben_rohrleitungen_data.keys(), angaben_rohrleitungen_data.values())

# Drucken des XML-str
xml_str = ET.tostring(formblatt_6_2, encoding='utf-8').decode()
print(xml_str)




<formblatt_6_2>&lt;?xml version="1.0" encoding="UTF-8"?&gt;<tabelle_kopf><ueberschrift>Angaben zum Formblatt</ueberschrift><spalten><spalte>Antragsunterlage</spalte><spalte>Anlage 1 / Formblatt 6.2</spalte></spalten><daten><zeile><wert>Für immissionsschutzrechtliches Genehmigungsverfahren</wert><wert>Detailangaben / Wassergefährdende Stoffe</wert></zeile></daten></tabelle_kopf><tabelle_angaben_anlage><ueberschrift>Angaben zur Anlage</ueberschrift><spalten><spalte>Anlagenart</spalte><spalte>Anlagenbezeichnung</spalte><spalte>Anlagenumfang</spalte></spalten><daten><zeile><wert>HBV-Anlage</wert><wert>C01V01-HEX.EX03</wert><wert>Rohrbündelwärmetauscher, Behälter, Plattenwärmetauscher, Rohrleitungssystem, Kolbenpumpe, Zentrifugalpumpe</wert></zeile></daten></tabelle_angaben_anlage><tabelle_angaben_stoffe><ueberschrift>Angaben zu den wassergefährdenden Stoffen</ueberschrift><spalten><spalte>Bezeichnung</spalte><spalte>Aggregatzustand</spalte><spalte>WGK</spalte><spalte>Volumen (m3) / Masse (

Formular PDF

In [ ]:
### Vorübergehend
stoff_bezeichnung = "Chemikalie A"
stoff_aggregatzustand = "Flüssig"
stoff_wgk = "1"
stoff_volumen_masse = "100 m3"
anlage_volumen = "500 m3"
anlage_wgk = "2"
anlage_gefaehrdungsstufe = "Stufe 2"
rl_bauart = "Typ X"
rl_irdisch = "Oberirdisch"
rl_anzahl = "3"
rl_material = "Kunststoff"
rl_zulassung = "Z-67890"


PDF mit latex // Pfad noch ändern

In [ ]:
from jinja2 import Template

# LaTeX-Template als String in einer Zelle
latex_template = r"""
\documentclass{article}
\usepackage[utf8]{inputenc}
\usepackage{enumitem}
\usepackage{fancyhdr}

% Seitenränder einstellen
\usepackage[left=2cm, right=2cm, top=2cm, bottom=1cm, headheight=1cm]{geometry}

% Fancyhdr-Stil definieren
\pagestyle{fancy}
\fancyhf{}  % Kopf- und Fußzeilen leeren

% Linie unter der Kopfzeile
\renewcommand{\headrulewidth}{0.4pt}

\begin{document}

\section*{Angaben zum Formblatt}

\subsection*{Antragsunterlage}

\begin{itemize}[label=--, left=0pt, itemsep=0pt]
    \item \textbf{Für immissionsschutzrechtliches Genehmigungsverfahren}
    \item \textbf{Detailangaben / Wassergefährdende Stoffe}
\end{itemize}

\subsection*{Angaben zur Anlage}

\begin{tabular}{|l|l|l|}
    \hline
    \textbf{Anlagenart} & \textbf{Anlagenbezeichnung} & \textbf{Anlagenumfang} \\
    \hline
    {{ anlagenart }} & {{ anlagenbezeichnung }} & {{ anlagenumfang }} \\
    \hline
\end{tabular}

\subsection*{Angaben zu den wassergefährdenden Stoffen}

\begin{tabular}{|l|l|l|l|}
    \hline
    \textbf{Bezeichnung} & \textbf{Aggregatzustand} & \textbf{WGK} & \textbf{Volumen (m3) / Masse (t)} \\
    \hline
    {{ stoff_bezeichnung }} & {{ stoff_aggregatzustand }} & {{ stoff_wgk }} & {{ stoff_volumen_masse }} \\
    \hline
\end{tabular}

\subsection*{Ermittlung der Gefährdungsstufe der Anlage nach §39 AwSV}

\begin{tabular}{|l|l|l|}
    \hline
    \textbf{Maßgebendes Volumen} & \textbf{Maßgebende WGK} & \textbf{Gefährdungsstufe} \\
    \hline
    {{ anlage_volumen }} & {{ anlage_wgk }} & {{ anlage_gefaehrdungsstufe }} \\
    \hline
\end{tabular}

\subsection*{Angaben zu Behältern}

\begin{tabular}{|l|l|l|l|l|}
    \hline
    \textbf{Tanknummer} & \textbf{Stoff} & \textbf{einwandig / doppelwandig} & \textbf{Nennvolumen} & \textbf{Material} & \textbf{Zulassung} \\
    \hline
    {{ tank_nummer }} & {{ tank_stoff }} & {{ tank_wandig }} & {{ tank_volumen }} & {{ tank_material }} & {{ tank_zulassung }} \\
    \hline
\end{tabular}

\subsection*{Angaben zu Rohrleitungen}

\begin{tabular}{|l|l|l|l|l|}
    \hline
    \textbf{Bauart} & \textbf{oberirdisch / unterirdisch} & \textbf{Anzahl} & \textbf{Material} & \textbf{Zulassung} \\
    \hline
    {{ rl_bauart }} & {{ rl_irdisch }} & {{ rl_anzahl }} & {{ rl_material }} & {{ rl_zulassung }} \\
    \hline
\end{tabular}

\end{document}
"""


# Erstellen einer Jinja2-Vorlage
template = Template(latex_template)

# Variablen setzen (diese sollten zuvor definiert worden sein)
variables = {
    "anlagenart": anlagenart,
    "anlagenbezeichnung": anlagenbezeichnung,
    "anlagenumfang": anlagenumfang,
    "stoff_bezeichnung": stoff_bezeichnung,
    "stoff_aggregatzustand": stoff_aggregatzustand,
    "stoff_wgk": stoff_wgk,
    "stoff_volumen_masse": stoff_volumen_masse,
    "anlage_volumen": anlage_volumen,
    "anlage_wgk": anlage_wgk,
    "anlage_gefaehrdungsstufe": anlage_gefaehrdungsstufe,
    "tank_nummer": tank["tank_nummer"],
    "tank_stoff": tank["tank_stoff"],
    "tank_wandig": tank["tank_wandig"],
    "tank_volumen": tank["tank_volumen"],
    "tank_material": tank["tank_material"],
    "tank_zulassung": tank["tank_zulassung"],
    "rl_bauart": rl_bauart,
    "rl_irdisch": rl_irdisch,
    "rl_anzahl": rl_anzahl,
    "rl_material": rl_material,
    "rl_zulassung": rl_zulassung,
}

# Rendern der Vorlage mit den Variablen
latex_code = template.render(variables)

# Pfad auf Desktop
desktop_path = os.path.join("/Users/millerfl/Desktop", "formular_6_2_auto.tex")

# Speichern des LaTeX-Codes in einer .tex-Datei auf dem Desktop
with open(desktop_path, "w") as tex_file:
    tex_file.write(latex_code)


NameError: name 'os' is not defined

Back-Up

PDF-Test simpel

In [ ]:
#from reportlab.pdfgen import canvas
#from reportlab.lib.pagesizes import letter

# Pfad zur PDF-Datei
#pdf_path = "/Users/millerfl/Desktop/test.pdf"

# Wort, das in die PDF geschrieben werden soll
#word_to_write = "Hello"

# Funktion zum Erstellen der PDF mit einem Wort
#def create_pdf(word, pdf_path):
    #pdf_canvas = canvas.Canvas(pdf_path, pagesize=letter)
    #pdf_canvas.drawString(100, 750, word)  # Position und Wort anpassen
    #pdf_canvas.save()

# Erstellen der PDF
#create_pdf(word_to_write, pdf_path)


PDF-Test erweitert

In [ ]:
# from reportlab.pdfgen import canvas
# from reportlab.lib.pagesizes import letter
# from reportlab.lib import colors
# from reportlab.platypus import Table, TableStyle
# import os
# import xml.etree.ElementTree as ET

# # Pfad zur PDF-Datei
# pdf_path = "/Users/millerfl/Desktop/formblatt_6_2.pdf"

# # Beispiel-XML-String (später ersetzen durch tatsächlichen XML-String)
# xml_str = """
# <formblatt_6_2>
#     <tabelle_angaben_anlage>
#         <daten>
#             <zeile>
#                 <wert>T1234</wert>
#                 <wert>ABC</wert>
#                 <wert>einwandig</wert>
#                 <wert>Metall</wert>
#                 <wert>DIN-Norm</wert>
#             </zeile>
#         </daten>
#     </tabelle_angaben_anlage>
# </formblatt_6_2>
# """

# # Erstellen des XML-Elements aus dem String
# formblatt_6_2 = ET.fromstring(xml_str)

# # Funktion zum Erstellen der PDF mit reportlab und alternativem Ansatz
# def create_pdf_alternative(xml_element, pdf_path):
#     try:
#         pdf_canvas = canvas.Canvas(pdf_path, pagesize=letter)
#         table_data = []

#         for child in xml_element.findall(".//zeile"):
#             row_data = [subchild.text for subchild in child]
#             table_data.append(row_data)

#         table_height = len(table_data) * 10
#         table_width = len(table_data[0]) * 100

#         # Setzen der Startposition für die Tabelle
#         x_offset, y_offset = 100, 750

#         # Erstellen der Table-Instanz und Hinzufügen der Daten
#         pdf_table = Table(table_data, colWidths=100, rowHeights=10)

#         # Definieren des Stils der Tabelle
#         style = TableStyle([('BACKGROUND', (0, 0), (-1, 0), colors.grey),
#                             ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
#                             ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
#                             ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
#                             ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
#                             ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
#                             ('GRID', (0, 0), (-1, -1), 1, colors.black)])

#         pdf_table.setStyle(style)

#         # Zeichnen der Tabelle auf den Canvas
#         pdf_table.wrapOn(pdf_canvas, 0, 0)
#         pdf_table.drawOn(pdf_canvas, x_offset, y_offset - table_height)

#         pdf_canvas.save()
#         print("PDF erfolgreich erstellt!")
#     except Exception as e:
#         print(f"Fehler beim Erstellen der PDF: {e}")

# # Erstellen des PDFs
# create_pdf_alternative(formblatt_6_2, pdf_path)


PDF-Test mit XSLT

In [ ]:
# GitHub Raw-URL für die XSLT
github_url_xslt = 'https://github.com/FloT10/MillerFlorinBA/blob/c60ce79783cdf3a9879b88448c7d054692c50ada/transform.xslt'

# Herunterladen der XSLT-Datei von der GitHub-URL.
response = requests.get(github_url_xslt)

# Speichern der heruntergeladenen XSLT-Datei lokal im Textmodus ('w').
with open('/tmp/transform.xslt', 'w') as file:
    file.write(response.text)

# Definieren eines Dateipfades zur lokal gespeicherten XSLT-Datei zur weiteren Verwendung.
transform_xslt_path = "/tmp/transform.xslt"

# Öffne die Datei und lese ihren Inhalt.
with open(transform_xslt_path, 'r') as file:
    transform_xslt_content = file.read()

# Gib den Inhalt der XSLT-Datei aus.
print(transform_xslt_content)

# Da Probleme mit Pfad vorübergehende Alternative
xslt_path = "/Users/millerfl/Desktop/transform.xslt"

{"payload":{"allShortcutsEnabled":false,"fileTree":{"":{"items":[{"name":".vscode","path":".vscode","contentType":"directory"},{"name":"C01V01-HEX.EX03.xml","path":"C01V01-HEX.EX03.xml","contentType":"file"},{"name":"LICENSE","path":"LICENSE","contentType":"file"},{"name":"README.md","path":"README.md","contentType":"file"},{"name":"formblatt_6.2.xml","path":"formblatt_6.2.xml","contentType":"file"},{"name":"prototyp.ipynb","path":"prototyp.ipynb","contentType":"file"},{"name":"transform.xslt","path":"transform.xslt","contentType":"file"}],"totalCount":7}},"fileTreeProcessingTime":1.263549,"foldersToFetch":[],"repo":{"id":733371200,"defaultBranch":"main","name":"MillerFlorinBA","ownerLogin":"FloT10","currentUserCanPush":false,"isFork":false,"isEmpty":false,"createdAt":"2023-12-19T07:14:43.000Z","ownerAvatar":"https://avatars.githubusercontent.com/u/143116830?v=4","public":true,"private":false,"isOrgOwned":false},"symbolsExpanded":false,"treeExpanded":true,"refInfo":{"name":"c60ce79783c

In [ ]:
import os

# Aktuelles Arbeitsverzeichnis anzeigen
current_directory = os.getcwd()
print("Aktuelles Arbeitsverzeichnis:", current_directory)


Aktuelles Arbeitsverzeichnis: /


In [ ]:
from lxml import etree

#Zurückgreifen auf vorhanden String
print(xml_str)

# Laden der XSLT-Datei
xslt_root = etree.parse(xslt_path)
transform = etree.XSLT(xslt_root)
xml_tree = etree.fromstring(xml_str)

result = transform(xml_tree)

#with open("output.html", "wb") as f:
#    f.write(result)

<formblatt_6_2>&lt;?xml version="1.0" encoding="UTF-8"?&gt;<tabelle_kopf><ueberschrift>Angaben zum Formblatt</ueberschrift><spalten><spalte>Antragsunterlage</spalte><spalte>Anlage 1 / Formblatt 6.2</spalte></spalten><daten><zeile><wert>Für immissionsschutzrechtliches Genehmigungsverfahren</wert><wert>Detailangaben / Wassergefährdende Stoffe</wert></zeile></daten></tabelle_kopf><tabelle_angaben_anlage><ueberschrift>Angaben zur Anlage</ueberschrift><spalten><spalte>Anlagenart</spalte><spalte>Anlagenbezeichnung</spalte><spalte>Anlagenumfang</spalte></spalten><daten><zeile><wert>HBV-Anlage</wert><wert>C01V01-HEX.EX03</wert><wert>Behälter, Plattenwärmetauscher, Kolbenpumpe, Rohrleitungssystem, Zentrifugalpumpe, Rohrbündelwärmetauscher</wert></zeile></daten></tabelle_angaben_anlage><tabelle_angaben_stoffe><ueberschrift>Angaben zu den wassergefährdenden Stoffen</ueberschrift><spalten><spalte>Bezeichnung</spalte><spalte>Aggregatzustand</spalte><spalte>WGK</spalte><spalte>Volumen (m3) / Masse (

OSError: Error reading file '/Users/millerfl/Desktop/transform.xslt': failed to load external entity "/Users/millerfl/Desktop/transform.xslt"

In [ ]:
import pdfkit

pdfkit.from_file("output.html", "output.pdf")


OSError: No such file: output.html

Test auf Zugriff 
(Ausführen im Terminal)
ls -l /Users/millerfl/Desktop/formblatt_6_2.pdf

Sonstiges

In [ ]:
# Später für Aufzählung
# Funktion zum Laden und Extrahieren von Daten aus der DEXPI-Datei.
def lade_daten(dexpi_xml):
    try:
        # Lade das XML-Dokument und erhalte die Wurzel des Baums.
        tree = ET.parse(dexpi_xml)
        root = tree.getroot()

        # Extrahiere die Anlagenbezeichnung aus dem XML-Baum.
        anlagenbezeichnung = root.find('.//Drawing').get('Name')

        # Extrahiere ComponentClass aus allen Equipment- und PipingNetworkSystem-Elementen.
        equipment_elements = root.findall('.//Equipment')
        piping_network_system_elements = root.findall('.//PipingNetworkSystem')

        component_classes = []

        for equipment_element in equipment_elements:
            component_class = equipment_element.get('ComponentClass')

            # Überprüfe, ob die ComponentClass den erlaubten Werten entspricht.
            erlaubte_component_classes = ['Tank', 'ReciprocatingPump', 'CentrifugalPump', 'PlateAndShellHeatExchanger', 'ShellAndTubeHeatExchanger']
            if component_class in erlaubte_component_classes:
                component_classes.append(component_class)

        for piping_network_system_element in piping_network_system_elements:
            component_class = piping_network_system_element.get('ComponentClass')

            # Überprüfe, ob die ComponentClass den erlaubten Werten entspricht.
            erlaubte_component_classes = ['PipingNetworkSystem']
            if component_class in erlaubte_component_classes:
                component_classes.append(component_class)

        # Überprüfe, ob mindestens eine gültige ComponentClass gefunden wurde.
        if not component_classes:
            raise ValueError("Keine gültige ComponentClass gefunden.")

        # Weitere Werte können entsprechend extrahiert werden.

        # Entferne Duplikate und erstelle ein eindeutiges Set von Komponentenklassen.
        unique_component_classes = set(component_classes)

        # Erstelle ein Dictionary mit dem Namen Anlagenumfang, in dem jede ComponentClass nur einmal vorkommt.
        Anlagenumfang = {klass: {} for klass in unique_component_classes}

        # Gib ein Dictionary mit den extrahierten Werten zurück.
        return {
            'anlagenbezeichnung': anlagenbezeichnung,
            'component_classes': component_classes,
            'Anlagenumfang': Anlagenumfang,
            # Weitere extrahierte Werte können hier hinzugefügt werden.
        }

    except Exception as e:
        # Falls ein Fehler auftritt, gib eine Fehlermeldung aus und gibt None zurück.
        print(f"Fehler beim Laden und Extrahieren der Daten: {e}")
        return None

# Beispielaufruf der Funktion und Ausgabe der extrahierten Daten.
daten = lade_daten(dexpi_xml)
print(daten)


{'anlagenbezeichnung': 'C01V01-HEX.EX03', 'component_classes': ['Tank', 'ReciprocatingPump', 'CentrifugalPump', 'ShellAndTubeHeatExchanger', 'PlateAndShellHeatExchanger', 'Tank', 'ReciprocatingPump', 'CentrifugalPump', 'ShellAndTubeHeatExchanger', 'PlateAndShellHeatExchanger', 'PipingNetworkSystem', 'PipingNetworkSystem', 'PipingNetworkSystem', 'PipingNetworkSystem', 'PipingNetworkSystem', 'PipingNetworkSystem', 'PipingNetworkSystem', 'PipingNetworkSystem', 'PipingNetworkSystem', 'PipingNetworkSystem', 'PipingNetworkSystem', 'PipingNetworkSystem', 'PipingNetworkSystem', 'PipingNetworkSystem'], 'Anlagenumfang': {'PipingNetworkSystem': {}, 'ReciprocatingPump': {}, 'PlateAndShellHeatExchanger': {}, 'ShellAndTubeHeatExchanger': {}, 'Tank': {}, 'CentrifugalPump': {}}}


In [ ]:
import requests
import xml.etree.ElementTree as ET

# GitHub Raw-URL für die DEXPI-R&I Datei
github_url_dexpi = 'https://raw.githubusercontent.com/FloT10/MillerFlorinBA/main/C01V01-HEX.EX03.xml'
# GitHub Raw-URL für die Formblatt-Vorlage
github_url_formblatt = 'https://raw.githubusercontent.com/FloT10/MillerFlorinBA/main/formblatt_6.2.xml'

# Dateien lokal speichern
def lade_und_speichere(url, lokal_path):
    response = requests.get(url)
    with open(lokal_path, 'w') as file:
        file.write(response.text)

# Laden und Speichern der DEXPI-R&I Datei
dexpi_xml = '/tmp/C01V01-HEX.EX03.xml'
lade_und_speichere(github_url_dexpi, dexpi_xml)

# Laden und Speichern der Formblatt-Vorlage
formblatt_vorlage = '/tmp/formblatt_6.2.xml'
lade_und_speichere(github_url_formblatt, formblatt_vorlage)

# Funktion zum Laden und Extrahieren der Werte aus dem DEXPI-R&I
def lade_daten(dexpi_xml):
    try:
        # Laden des DEXPI-R&I Dokuments
        tree = ET.parse(dexpi_xml)
        root = tree.getroot()

        # Extrahieren der Werte, wie benötigt
        anlagenbezeichnung = root.find('.//Drawing').get('Name')
        # Weitere Werte entsprechend extrahieren

        # Kontrollausgabe der Anlagenbezeichnung
        print(f"Anlagenbezeichnung zur Kontrolle: {anlagenbezeichnung}")

        return {
            'anlagenbezeichnung': anlagenbezeichnung,
            # Weitere extrahierte Werte hinzufügen
        }

    except Exception as e:
        print(f"Fehler beim Laden und Extrahieren der Daten aus DEXPI-R&I: {e}")
        return None

# Funktion zum Befüllen der Formblatt-Vorlage mit den extrahierten DEXPI-R&I Daten
def befuellen_und_speichern(formblatt_vorlage, formblatt_befüllt, daten):
    try:
        tree = ET.parse(formblatt_vorlage)
        root = tree.getroot()

        # Befülle die Vorlage mit den extrahierten Daten
        root.find('.//anlagenbezeichnung').text = daten['anlagenbezeichnung']
        # Weitere Werte in der Vorlage entsprechend befüllen

        # Speichere die befüllte Vorlage als neue Datei
        tree.write(formblatt_befüllt)

        print(f"Das Formblatt wurde erfolgreich mit den Daten befüllt und unter {formblatt_befüllt} gespeichert.")

    except Exception as e:
        print(f"Fehler beim Befüllen und Speichern des Formblatts: {e}")

# Lade Daten aus der DEXPI-R&I Datei
daten_dexpi = lade_daten(dexpi_xml)

# Lade die Formblatt-Vorlage
formblatt_befüllt= '/tmp/Ziel/Formblatt_befüllt.xml'

# Befülle die Formblatt-Vorlage und speichere das Ergebnis
befuellen_und_speichern(formblatt_vorlage, formblatt_befüllt, daten_dexpi)


Anlagenbezeichnung zur Kontrolle: C01V01-HEX.EX03
Fehler beim Befüllen und Speichern des Formblatts: mismatched tag: line 233, column 2
